#### Noisy Quantum SVM - New Try

In [312]:
import qiskit, qiskit_aer, qiskit_machine_learning
print("Qiskit:", qiskit.__version__)
print("Aer:", qiskit_aer.__version__)
print("QML:", qiskit_machine_learning.__version__)

Qiskit: 1.4.4
Aer: 0.17.2
QML: 0.8.4


In [313]:
# To ensure reproducibility of results
from qiskit_machine_learning.utils import algorithm_globals
algorithm_globals.random_seed = 12345

In [314]:
# Imports
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Qiskit and Qiskit Aer import
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error

from qiskit_aer.primitives import SamplerV2 as AerSampler
from qiskit.circuit.library import ZZFeatureMap

# Qiskit machine learning
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC

# Getting realistic noise model
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit_ibm_runtime import QiskitRuntimeService

# For transpilation
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


In [316]:
# Load data first
lung_cancer_column_names = ['label'] + [f'attr_{i}' for i in range(1, 57)]
file_path_lung = r'C:\Users\User\Documents\MyProjects\FYP_ResearchProject\data\lung+cancer\lung-cancer.data'

# reads the data, treating "?" as missing values
df_lung = pd.read_csv(file_path_lung, header=None, names=lung_cancer_column_names, na_values=['?'])

print(f"Original shape of Lung Cancer data: {df_lung.shape}")

Original shape of Lung Cancer data: (32, 57)


In [317]:
# Mode imputation for missing values
modes = df_lung.mode().iloc[0]
df_lung.fillna(modes, inplace=True)

# Then check if all Nan are gone
print(f"Total missing values after imputation: {df_lung.isnull().sum().sum()}\n")

Total missing values after imputation: 0



In [318]:
# Separate features and target variable
X_lung = df_lung.drop('label', axis=1)
y_lung = df_lung['label']

In [319]:
# Target binarization
# y_lung_binary = y_lung.apply(lambda x: 1 if x == 'M' else 0)
y_lung_binary = y_lung.apply(lambda x: 0 if x == 1 else 1)

In [320]:
# Data splitting 
X_train_lc, X_test_lc, y_train_lc, y_test_lc = train_test_split(
    X_lung, y_lung_binary, test_size=0.3, random_state=42, stratify=y_lung_binary
)

In [321]:
print(X_train_lc.shape)
print(X_test_lc.shape)

(22, 56)
(10, 56)


In [322]:
# Scaling and PCA process
scaler_lc = StandardScaler()
X_train_lc_scaled = scaler_lc.fit_transform(X_train_lc)
X_test_lc_scaled = scaler_lc.transform(X_test_lc) # use transform here, not fit_transform

n_components = 4
pca = PCA(n_components=n_components)
X_train_lc_pca = pca.fit_transform(X_train_lc_scaled)
X_test_lc_pca = pca.transform(X_test_lc_scaled)

print(f"Data preprocessed. Training set shape: {X_train_lc_pca.shape}")
print(f"Test set shape: {X_test_lc_pca.shape}\n")


Data preprocessed. Training set shape: (22, 4)
Test set shape: (10, 4)



##### Setup Noise Model 

In [ ]:
# Depolarizing Error
noise_model = NoiseModel()
error_prob = 0.05
depol_error = depolarizing_error(error_prob, 1)
noise_model.add_all_qubit_quantum_error(depol_error, ['u1', 'u2', 'u3', 'rx', 'ry', 'rz', 'id'])


In [ ]:
# Create Noise Sampler
noise_sampler = AerSampler(
    options={
        "backend_options": {
            "noise_model": noise_model
        }
    }
)

##### Quantum Kernel Implementation

In [ ]:
# Using Base AerSimulator
aer_backend = AerSimulator()

# Creating a TRANSPILATION MANAGER
# This ensures all circuits are converted into the native gates (basis gates)
# the simulator and noise model use.
# optimization_level=1 is will be a good starting point for balancing speed and circuit quality.
pm = generate_preset_pass_manager(optimization_level=1, backend=aer_backend)

# Reinstantiate the ComputeUncompute and then passing the pass_manager
fidelity = ComputeUncompute(sampler=noise_sampler, pass_manager=pm)
qkernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=fm)
qsvc_noisy = QSVC(quantum_kernel=qkernel)

In [ ]:
# PLEASE LA DAPATTTTT 
print ("Calculating training kernel matrix ...")
start_time = time.time()
qsvc_noisy.fit(X_train_lc_pca, y_train_lc)
end_time = time.time()
print(f"QSVC training finished in {end_time - start_time:.2f} seconds.")

Calculating training kernel matrix ...
QSVC training finished in 2.09 seconds.


In [ ]:
y_pred_noisy = qsvc_noisy.predict(X_test_lc_pca)
accuracy_noisy = accuracy_score(y_test_lc, y_pred_noisy)

print("\n--- Noisy QSVC Evaluation Results ---")
print(f"Accuracy Score: {accuracy_noisy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test_lc, y_pred_noisy))


--- Noisy QSVC Evaluation Results ---
Accuracy Score: 0.7000

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.70      1.00      0.82         7

    accuracy                           0.70        10
   macro avg       0.35      0.50      0.41        10
weighted avg       0.49      0.70      0.58        10



c:\Users\User\anaconda3\envs\qsvm_conda\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\anaconda3\envs\qsvm_conda\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\anaconda3\envs\qsvm_conda\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", 